In [267]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [268]:
from rlway.pyosrd import OSRD

In [269]:
OSRD.use_cases

['cvg_dvg', 'point_switch', 'station_capacity2', 'straight_line']

In [270]:
use_case='straight_line'
sim = OSRD(use_case=use_case, dir='tmp')

[12:44:16,935] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[12:44:16,965] [INFO]              [FullInfra] parsing infra
[12:44:17,027] [INFO]              [FullInfra] adaptation to kotlin
[12:44:17,055] [INFO]              [FullInfra] loading signals
[12:44:17,061] [INFO]              [FullInfra] building blocks
[12:44:17,066] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[12:44:17,080] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.68
[12:44:17,128] [DEBUG]             [Allowance] computing range n°1
[12:44:17,130] [DEBUG]             [Allowance]   computing section n°1
[12:44:17,130] [DEBUG]             [Allowance]   target time = 210.525
[12:44:17,130] [DEBUG]             [Allowance]     starting attempt 1
[12:44:17,132] [DEBUG]             [Allowance]     envelope time 336.339
[12:44:17,132] [DEBUG]             [Allowance]     starting attempt 2
[12:44:17,132] [DEBUG]             [Allowance]     

In [271]:
sim.train_track_sections(0)

[{'id': 'T', 'direction': 'START_TO_STOP'}]

In [272]:
sim.space_time_chart_plotly(0, points_to_show=['station',], eco_or_base='eco')